In [ ]:
from pprint import pprint
import requests
from bs4 import BeautifulSoup
from htmldate import find_date
import numpy as np
import random as rd
from lxml.html import fromstring
from itertools import cycle
import time
from datetime import datetime
import os
from googleapiclient.discovery import build 

In [ ]:
from googlesearch import search

In [ ]:
NUM_OF_RESULTS = 20
DATA_DIR = 'data'
OUT_DIR = 'out'
AUTH_DIR = 'C:\\PycharmProjects\\universities_covid19_auth.txt'
sep = os.sep

In [ ]:
universities = [u.strip() for u in open(f'{DATA_DIR}{sep}Universities.txt').readlines()]

In [ ]:
api_key, cse_id = [l.split('=')[1].strip() for l in open(AUTH_DIR).readlines()]

In [ ]:
def google_query(query, api_key, cse_id, **kwargs):
    query_service = build("customsearch", 
                          "v1", 
                          developerKey=api_key
                          )  
    query_results = query_service.cse().list(q=query,    # Query
                                             cx=cse_id,  # CSE ID
                                             **kwargs    
                                             ).execute()
    return query_results['items']

In [ ]:
universitites_search = {}
for uni in universities:
    domain_qry = f'{uni} (Home | Home Page)'
    qry_results = google_query(domain_qry, api_key, cse_id, num = NUM_OF_RESULTS)
    link0 = qry_results[0]['link'].replace('www.', '')
    uni_domain = link0[link0.find('://'):link0.find('.')][3:]
    
    qry = f'inurl:{uni_domain} {uni} (COVID-19| COVID 19| COVID19 | Coronavirus 2019)'
    my_results = google_query(qry, api_key, cse_id, num = 10)
    
    universitites_search[uni] = []
    for result in my_results:
        universitites_search[uni].append(result['link'])

In [ ]:
for uni, urls in universitites_search.items():
    with open(f'{OUT_DIR}{os.sep}{uni}.csv', 'w', "utf-8") as wr:
        for url in urls:
            try:
                url = url.split('#')[0].replace('www.', '')

                page = requests.get(url, timeout=5)
                if page.status_code != 200:
                    continue

                soup = BeautifulSoup(page.content, 'html.parser')

                try:
                    h = soup.find('h1').text.strip()
                except:
                    h = soup.find('title').text.strip()

                if h is None or h.strip() == '':
                    h = 'COVID-19 Updates.'

                date = find_date(url)

                line = f"{url} , {h} , {date if date else datetime.today().strftime('%Y-%m-%d')}"

                print(line)
                wr.write(line + '\n')
                wr.flush()
            except:
                print('### ERROR! ',url)

In [ ]:
len(universitites_search)

In [ ]:
# def get_domain(uni):
#     domain_qry = f'{uni} (Home | Home Page)'
#     for url in search(domain_qry,        # The query you want to run
#                 tld = 'com',  # The top level domain
#                 lang = 'en',  # The language
#                 num = 1,     # Number of results per page
#                 start = 0,    # First result to retrieve
#                 stop = 1,  # Last result to retrieve
#                 pause = rd.choice(delays),  # Lapse between HTTP requests
#                 user_agent=rd.choice(user_agents)
#                ):
#         url = url.replace('www.', '')
#         return url[url.find('://'):url.find('.')][3:]

In [ ]:
# for uni in universities:
#     with open(f'{OUT_DIR}{os.sep}{uni}.csv', 'w') as wr:
#         domain = get_domain(uni)
#         qry = f'inurl:{domain} {uni} (COVID-19| COVID 19| COVID19 | Coronavirus 2019)'
#         print(f'##### inurl:{domain} {uni} #######################')
        
#         for url in search(qry,        # The query you want to run
#                     tld = 'com',  # The top level domain
#                     lang = 'en',  # The language
#                     num = 5,     # Number of results per page
#                     start = 0,    # First result to retrieve
#                     stop = 10,  # Last result to retrieve
#                     pause = rd.choice(delays),  # Lapse between HTTP requests
#                     user_agent=rd.choice(user_agents)
#                    ):
#             try:
#                 url = url.split('#')[0].replace('www.', '')
                

#                 page = requests.get(url, timeout=5)
#                 if page.status_code != 200:
#                     continue

#                 soup = BeautifulSoup(page.content, 'html.parser')

#                 title = soup.find('title').text.strip()
#                 h1 = soup.find('h1').text.strip()
#                 date = find_date(url)

#                 line = f"{url} , {title} , {date if date else datetime.today().strftime('%Y-%m-%d')}"

#                 wr.write(line + '\n')
#                 wr.flush()

#                 print(line)
#                 time.sleep(rd.choice(delays))

#             except Exception as e:
#                 print('Error:', url, e)